Susikeliam naudojamus paketus

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pickle, gzip
import numpy as np
import torchvision.transforms as transforms
from google.colab import files



In [ ]:
uploaded = files.upload()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


Susitvarkome duomenų formatą. Taip pat atliekame kelias posūkio, vertimo, spalvos ir padidinimo transformacijas turimiems duomenims ir pridedame prie esamų

In [2]:
import random

with gzip.open('/content/drive/MyDrive/Colab Notebooks/flatland_train.data', 'rb') as f:
    X, y = pickle.load(f)

X = X / 255.0

X = X.reshape(-1, 1, 50, 50)


y[y != 0] -= 2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)


class ShapeDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label
    def shape(self):
        return torch.Size([len(self.images)] + list(self.images[0].shape))

augmented_images = []
augmented_labels = []

augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomVerticalFlip(p=1),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
    transforms.ColorJitter(brightness=0.3, contrast=0.3)
])


for i in range(len(X_train)):
    original_image = X_train[i]
    label = y_train[i]

    augmented_images.append(original_image)
    augmented_labels.append(label)

    transformed_image = augmentation_transforms(transforms.ToPILImage()(original_image))
    transformed_image = transforms.ToTensor()(transformed_image)

    augmented_images.append(transformed_image)
    augmented_labels.append(label)

augmented_images = torch.stack(augmented_images)
augmented_labels = torch.tensor(augmented_labels, dtype=torch.long)

X_train = augmented_images
y_train = augmented_labels

train_dataset = ShapeDataset(X_train, y_train, transform=None)
test_dataset = ShapeDataset(X_test, y_test, transform=None)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"New Training data shape: {train_dataset.shape()}")
print(f"New Training data shape: {test_dataset.shape()}")


New Training data shape: torch.Size([14000, 1, 50, 50])
New Training data shape: torch.Size([3000, 1, 50, 50])


In [3]:
class ImprovedShapeClassifier(nn.Module):
    def __init__(self):
        super(ImprovedShapeClassifier, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.gap = nn.AdaptiveAvgPool2d(1)

        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 5)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)

        x = self.gap(x)
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [ ]:
# Initialize model, loss function, and optimizer
model = ImprovedShapeClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # weight_decay adds L2 regularization
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # Decreases learning rate by 0.1 every 10 epochs

# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        running_loss += loss.item()
     # Step the scheduler
    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'shape_classifier.pth')


Epoch [1/15], Loss: 0.5502523226008568
Epoch [2/15], Loss: 0.22789788844922906
Epoch [3/15], Loss: 0.1592528848870568
Epoch [4/15], Loss: 0.13831854531686055
Epoch [5/15], Loss: 0.1294964459431294
Epoch [6/15], Loss: 0.11774667229112153
Epoch [7/15], Loss: 0.10513983568661425
Epoch [8/15], Loss: 0.10427386822714789
Epoch [9/15], Loss: 0.10384256289378829
Epoch [10/15], Loss: 0.08891717818901623
Epoch [11/15], Loss: 0.058068999644266824
Epoch [12/15], Loss: 0.0485492148457869
Epoch [13/15], Loss: 0.04300375570028544
Epoch [14/15], Loss: 0.03712638155621004


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Test Accuracy: 98.36666666666666%


In [ ]:
import pickle, gzip

with gzip.open('/content/drive/MyDrive/Colab Notebooks/flatland_test.data', 'rb') as f:
    X_t, y_test = pickle.load(f)

X_t = X_t / 255.0

X_t = X_t.reshape(-1, 1, 50, 50)
X_t = torch.tensor(X_t, dtype=torch.float32)

model.eval()
predicted_classes = []
batch_size = 256

with torch.no_grad():
    for i in range(0, len(X_t), batch_size):
        batch_X = X_t[i:i + batch_size]
        batch_predictions = model(batch_X)
        batch_predicted_classes = torch.argmax(batch_predictions, dim=1).cpu().numpy()
        predicted_classes.extend(batch_predicted_classes)


predicted_classes = np.array(predicted_classes)
predicted_classes[predicted_classes != 0] += 2


result_string = ''.join(map(str, predicted_classes))
with open('predicted_classes.txt', 'w') as f:
    f.write(result_string)

print("Predicted classes saved to 'predicted_classes.txt'")


Predicted classes saved to 'predicted_classes.txt'


In [ ]:
from google.colab import files
files.download('predicted_classes.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>